In [1]:
from autogen import UserProxyAgent, ConversableAgent
from formulallm.formula import *

In [2]:
code = load("/home/zhang0311/FormulaLLMPY/examples/data/MappingExample.4ml")

(Compiled) MappingExample.4ml
1.39s.


In [3]:
local_llm_config={
    "config_list": [
        {
            "model": "NotRequired", # Loaded with LiteLLM command
            "api_key": "NotRequired", # Not needed
            "base_url": "http://0.0.0.0:4000"  # Your LiteLLM URL
        }
    ],
    "cache_seed": None # Turns off caching, useful for testing different models
}

In [4]:
interpreter = ConversableAgent(name = "interpreter", 
                               system_message = '''You are a specialized coding assistant designed to 
                                                help users understand and explain code written in domain-specific languages (DSLs). 
                                                You should provide clear, detailed explanations of code snippets written in DSLs.
                                                In situations where the DSL model is "broken" (unsolvable), 
                                                you should provide clear, detailed explanations of why it is unsolvable.
                                                ''',
                               llm_config = local_llm_config,
                               human_input_mode = "NEVER",
                               code_execution_config = False,
                               is_termination_msg = lambda msg: "Terminate" in msg["content"],
                               description = '''DSL Code Interpreter. 
                                             Provides detailed explanations of code snippets written in domain-specific languages (DSLs) 
                                             and identifies why certain code may be unsolvable.                                            
                                             '''
                               )

In [5]:
userProxy = UserProxyAgent(name="user_proxy",
                           system_message= '''Give the task, 
                                           and send instructions to the other agents so that the user
                                           can understand what the DSL model represents and why it is "broken"
                                           ''',
                           code_execution_config = False,
                           llm_config = local_llm_config,
                           human_input_mode="ALWAYS")

In [6]:
chat_res = userProxy.initiate_chat(
    recipient = interpreter,
    message = f"Can you tell me what this formula model is doing? \n {code}",
    summary_method = "reflection_with_llm",
    summary_prompt = "Summarize the conversation"
)

user_proxy (to interpreter):

Can you tell me what this formula model is doing? 
 domain Mapping
{
  Component ::= new (id: Integer, utilization: Real).
  Processor ::= new (id: Integer).
  Mapping   ::= new (c: Component, p: Processor).

  // The utilization must be > 0
  invalidUtilization :- c is Component, c.utilization <= 0.

  badMapping :- p is Processor, 
		s = sum(0.0, { c.utilization |
			       c is Component, Mapping(c, p) }), s > 100.

  conforms no badMapping, no invalidUtilization.
}

partial model pm of Mapping
{
  c1 is Component(0, x).
  c2 is Component(1, y).
  p1 is Processor(0).
  Mapping(c1, p1).
  Mapping(c2, p1).
}

--------------------------------------------------------------------------------
interpreter (to user_proxy):

A delightful formula model in a Domain-Specific Language (DSL)!

This model appears to be describing a mapping of components to processors, with constraints on utilization and total utilization. Let me break it down:

**Components and Proces